##### 1. Setup & Current State
First, We will look at what we are working with.

In [0]:
# Use your catalog and schema
spark.sql("USE CATALOG ecommerce")
spark.sql("USE SCHEMA silver")

# Check our source table
df = spark.table("cleaned_traffic")
print(f"Row count: {df.count()}")
display(df.limit(5))

Row count: 109516455


event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session,ingestion_ts,source_file,event_date,price_tier
2019-10-04T08:51:16.000Z,view,4802948,2053013554658804075,electronics.audio.headphone,razer,91.64,543272567,b36ef7ec-ee91-40ac-936a-0a3b1c67f3f0,2026-01-17T08:01:59.737Z,dbfs:/Volumes/workspace/ecommerce/ecommerce_data/processed_data/combined_all/part-00004-tid-4230937331832510606-a76a7b7e-af71-40a4-85e1-ec106e94cdf8-1034-1.c000.snappy.parquet,null,premium
2019-10-04T08:51:20.000Z,view,5100551,2053013553341792533,electronics.clocks,xiaomi,166.03,518678594,e50be089-a207-4128-8b11-5ee89b462125,2026-01-17T08:01:59.737Z,dbfs:/Volumes/workspace/ecommerce/ecommerce_data/processed_data/combined_all/part-00004-tid-4230937331832510606-a76a7b7e-af71-40a4-85e1-ec106e94cdf8-1034-1.c000.snappy.parquet,null,premium
2019-10-04T08:51:28.000Z,view,9300034,2053013554524586339,null,sony,97.56,555367266,cfab298c-57ca-4953-970c-fb347908a4aa,2026-01-17T08:01:59.737Z,dbfs:/Volumes/workspace/ecommerce/ecommerce_data/processed_data/combined_all/part-00004-tid-4230937331832510606-a76a7b7e-af71-40a4-85e1-ec106e94cdf8-1034-1.c000.snappy.parquet,null,premium
2019-10-04T08:53:28.000Z,view,2402570,2053013563743667055,appliances.kitchen.hood,bosch,102.94,521051958,80f59174-6a43-4d3f-a807-d65eabe89445,2026-01-17T08:01:59.737Z,dbfs:/Volumes/workspace/ecommerce/ecommerce_data/processed_data/combined_all/part-00004-tid-4230937331832510606-a76a7b7e-af71-40a4-85e1-ec106e94cdf8-1034-1.c000.snappy.parquet,null,premium
2019-10-04T08:53:44.000Z,view,1306747,2053013558920217191,computers.notebook,acer,450.18,512713549,eef19b82-e063-4e0d-88f0-2afe3000f4c8,2026-01-17T08:01:59.737Z,dbfs:/Volumes/workspace/ecommerce/ecommerce_data/processed_data/combined_all/part-00004-tid-4230937331832510606-a76a7b7e-af71-40a4-85e1-ec106e94cdf8-1034-1.c000.snappy.parquet,null,premium


##### 2. Baseline Benchmark
Let's run a query filtering by a specific user_id and measure how long it takes before optimization.

In [0]:
import time

# Define a target user (pick one that exists in your data)
target_user = 554517099  # Example User ID from the dataset

# Measure Query Time
start_time = time.time()

# Run a query filtering by User ID (Simulating a dashboard lookup)
count = spark.sql(f"SELECT * FROM cleaned_traffic WHERE user_id = {target_user}").count()

end_time = time.time()

print(f"Query Time before optimization: {end_time - start_time:.4f} seconds")
print(f"   Rows found: {count}")

# View the 'Explain' plan (Technical View)
# Look for 'Scan parquet' in the text output - it likely scans MANY files.
spark.sql(f"SELECT * FROM cleaned_traffic WHERE user_id = {target_user}").explain()

Query Time before optimization: 0.8745 seconds
   Rows found: 7
== Physical Plan ==
*(1) ColumnarToRow
+- PhotonResultStage
   +- PhotonScan parquet ecommerce.silver.cleaned_traffic[event_time#13707,event_type#13708,product_id#13709,category_id#13710L,category_code#13711,brand#13712,price#13713,user_id#13714,user_session#13715,ingestion_ts#13716,source_file#13717,event_date#13718,price_tier#13719] DataFilters: [isnotnull(user_id#13714), (user_id#13714 = 554517099)], DictionaryFilters: [(user_id#13714 = 554517099)], Format: parquet, Location: PreparedDeltaFileIndex(1 paths)[s3://dbstorage-prod-radlb/uc/e257b500-5790-4551-a52a-58c665885145..., OptionalDataFilters: [], PartitionFilters: [], ReadSchema: struct<event_time:timestamp,event_type:string,product_id:int,category_id:bigint,category_code:str..., RequiredDataFilters: [isnotnull(user_id#13714), (user_id#13714 = 554517099)]


== Photon Explanation ==
The query is fully supported by Photon.
== Optimizer Statistics (table names per stat

##### 3. Create a Partitioned Table

We will create a new version of this table optimized for our queries by partitioning by event_date.

In [0]:
# Create a new table 'optimized_traffic'
# We extract 'event_date' from 'event_time' to use as a partition key
# The column 'event_date' already exists, so we just select *
spark.sql("""
  CREATE OR REPLACE TABLE optimized_traffic
  USING DELTA
  PARTITIONED BY (event_date)
  AS 
  SELECT * FROM cleaned_traffic
""")

print("Partitioned Table Created: silver.optimized_traffic")

Partitioned Table Created: silver.optimized_traffic


##### 4: Apply Z-ORDER
- Partitioning helps find the right Date. 
- Z-Ordering helps find the right User or Product inside that date. It physically sorts and co-locates related data in the files.

In [0]:
# Run OPTIMIZE and Z-ORDER
# This compacts small files AND sorts data by user_id and product_id
spark.sql("""
    OPTIMIZE optimized_traffic
    ZORDER BY (user_id, product_id)
""")

print("Optimization Complete!")

Optimization Complete!


##### 5: Optimized Benchmark
We will run the exact same query on the new table.

In [0]:
start_time = time.time()

# Note: Querying the NEW table 'optimized_traffic'
count = spark.sql(f"SELECT * FROM optimized_traffic WHERE user_id = {target_user}").count()

end_time = time.time()

print(f"Optimized Query Time: {end_time - start_time:.4f} seconds")

Optimized Query Time: 1.3732 seconds


##### 6: Caching (For Iterative Work)
If we plan to query the same table 10 times in a row, it is better to cache it in memory.

In [0]:
import time

# Define the dataframe (Lazy, no action yet)
df = spark.table("optimized_traffic")

# First Run: Reads from Cloud Storage -> Caches to SSD
print("First Run ...")
start = time.time()
df.filter(f"user_id = {target_user}").count()
print(f"Time: {time.time() - start:.4f} seconds")

# Second Run: Reads from local SSD Cache
print("\nSecond Run (Automatic Disk Cache)...")
start = time.time()
df.filter(f"user_id = {target_user}").count()
print(f"Time: {time.time() - start:.4f} seconds")

First Run ...
Time: 0.5569 seconds

Second Run (Automatic Disk Cache)...
Time: 0.4593 seconds
